# Programming for Deep Reinforcement Learning

Lab Course: Challenging Problems in Reinforcement Learning

In this notebook, we focus on two things that can be useful to your project work and that you will encounter a lot when programming for (Deep) Reinforcement Learning (RL) -- and also general Machine Learning.
1. Implementing a Deep RL algorithm in ```Pytorch```
2. Using Deep RL algorithms from the ```stable_baselines3``` library

Additionally, for those interested in safety in RL, we include
3. Safe RL training using ```omnisafe``` and ```safety_gymnasium```

Please fill out the code everywhere it says `# TODO: ...`.

## Installing the relevant libraries

Run the cell below if you don't have all necessary libraries installed.

In [ ]:
!pip install torch -q
!pip install gymnasium -q
!pip install stable-baselines3 -q
!pip install omnisafe -q
!pip install safety-gymnasium -q

## Deep Q-Learning in Pytorch

In the first notebook, we used (standard, tabular) Q-learning on two gymnasium environments. When we combine Q-learning with neural networks to represent the policy, the resulting algorithm is called DQN (Deep Q-learning). DQN is one of the most fundamental and simple algorithms in DRL. Please check the original paper [here](https://arxiv.org/pdf/1312.5602) for details on how it works!

We can implement DQN (and in general, neural network based Machine Learning + RL algorithms) easily in ```pytorch```. ```pytorch``` is one of the major frameworks used for this task (there exist more e.g. Tensorflow, JAX), so understanding how to use it is a good skill to have if you like ML!

In this part of the notebook, we use ```pytorch``` to implement DQN. Disclaimer: DQN is a fundamental, well-known algorithm. There are many implementations available online. Therefore, we largely use code from [this](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) pytorch tutorial here. The purpose is simply to introduce you to ML/RL coding in ```pytorch```.

Additionally, we include the pseudo-code of DQN, taken from the original paper (linked above), here. Take a look and compare it to the tabular Q-learning you used last week! The major differences here lie in the use of a replay memory to store observations in the form of tuples (state, action, next state, reward), that the model is trained on, and in the use of (two) neural networks to represent the policy, instead of a Q-table.

[<img src="DQN_algorithm.png" width="750"/>](DQN_algorithm.png)

In [4]:
# First, we import the necessary libraries
import gymnasium as gym
import math
import random
import matplotlib.pyplot as plt
from IPython import display
from collections import namedtuple, deque

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Then, like last week, we create our gym environment.
# TODO: create an environment for the "CartPole-v1" task!
""" START COCE """

""" END CODE """

In [6]:
# The replay memory (or replay buffer) stores transitions in the form of (state, action, next_state, reward)
# In every step, the current transition is added to the replay memory.
# During learning, batches of transition are sampled from the memory and used to update the policy
# The replay memory is not specific to DQN, but used in many DRL algorithms.
# We provide you with the data structure here.

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, state, action, next_state, reward):
        """Save a transition"""
        self.memory.append(Transition(state, action, next_state, reward))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [7]:
# This class is used for the neural network representation of the Q function.
# It is based on the pytorch class nn.Module - this class is the basis of all Pytorch neural network classes.
# It needs at least two main functions: __init__ is used to initialise the layers of the network,
# and forward "forwards" the input through the layers of the network:
# It takes the state of the RL environment as input, and predicts the next action the agent should take.

# Disclaimer: We renamed this class from the Pytorch Tutorial, from DQN to QNetwork, since it does not represent the entire algorithm, only the Q function!

class QNetwork(nn.Module):

    def __init__(self, n_observations, n_actions, n_hidden):
        super(QNetwork, self).__init__()
        self.layer1 = nn.Linear(n_observations, n_hidden)
        self.layer2 = nn.Linear(n_hidden, n_hidden)
        # TODO: add the third (output) layer using the right dimensions!
        """ START CODE """

        """ END CODE """

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [8]:
# We use this function to plot the episode reward during and after training.
episode_rewards = []

def plot_rewards(show_result=False):
    plt.figure(1)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.plot(episode_rewards)

    plt.pause(0.001)
    if not show_result:
        display.display(plt.gcf())
        display.clear_output(wait=True)
    else:
        display.display(plt.gcf())

In [9]:
# Now we will set some hyperparameters for training
# Feel free to change values and watch the observe the effect they have on training!

BATCH_SIZE = 128        # the number of transitions sampled from the replay buffer
GAMMA = 0.99            # the discount factor
EPS_START = 0.9         # the value of epsilon at the beginning of training
EPS_END = 0.05          # the final value of epsilon at the end of training
EPS_DECAY = 1000        # controls the rate of exponential decay of epsilon, higher means a slower decay
TAU = 0.005             # update rate of the target network
LR = 1e-4               # the learning rate
N_HIDDEN = 128          # number of neurons in the hidden layer -> check out the QNetwork class!
MEMORY_CAPACITY = 10000 # size of the replay memory

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

NUM_EPISODES = 500

In [ ]:
# Now, we implement the DQN algorithm itself, following the pseudo-code from the original paper.

## Algorithm 1 Deep Q-learning with Experience Replay

## Initialize replay memory D to capacity N
memory = ReplayMemory(MEMORY_CAPACITY)
## Initialize action-value function Q with random weights
q_network = QNetwork(n_observations, n_actions, N_HIDDEN)
# TODO: initialize a target network with the same dimensions as the q_network!
""" START CODE """

""" END CODE """
target_net.load_state_dict(q_network.state_dict())

optimizer = torch.optim.AdamW(q_network.parameters(), lr=LR, amsgrad=True)

steps_done = 0

## for episode 1, M do
for i_episode in range(NUM_EPISODES):
    episode_reward = 0
##   Initialise sequence s_1 = {x_1} and preprocessed sequenced phi_1 = phi(s_1)
    # TODO: get the initial state from the environment using the reset() function
    """ START CODE """

    """ END CODE """
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
##   for t = 1, T do
    while True:
##       With probability epsilon select a random action a_t
##       otherwise select a_t = max_a Q*(phi(s_t), a; theta)
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        # TODO: implement epsilon-greedy - remember from the previous notebook, or look at the pseudo-code!
        # For the random action, use this: torch.tensor([[env.action_space.sample()]], dtype=torch.long)
        # For the action based on the policy, use this: q_network(state).max(1).indices.view(1,1)
        """ START CODE """


        """ END CODE """
##       Execute action a_t in emulator and observe reward r_t and image x_{t+1}
##       Set s_{t+1} = s_t, a_t, x_{t+1} and preprocess phi_{t+1} = phi(s_{t+1})

        # TODO: using the action.item(), take a step in the environment and observe the observation, reward, terminated, truncated, _.
        """ START CODE """

        """ END CODE """
        episode_reward += reward
        reward = torch.tensor([reward])
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32).unsqueeze(0)

##       Store transition (phi_t, a_t, r_t, phi_{t+1}) in D
        # TODO: add the relevant information to the replay buffer!
        """ START CODE """

        """ END CODE """
        state = next_state

        if len(memory) >= BATCH_SIZE:
##       Sample random minibatch of transitions (phi_j, a_j, r_j, phi_{j+1}) from D
            # TODO: sample a batch of transitions of size BATCH_SIZE from the memory
            """ START CODE """

            """ END CODE """
            batch = Transition(*zip(*transitions))
##       Set y_j =   {r_j, for terminal phi_{j+1}
##                   {r_j + gamma * max_a' Q(phi_{j+1}, a'; theta), for non-terminal phi_{j+1}
##       Perform a gradient descent step on (y_j - Q(phi_j, a_j; theta))**2 according to equation 3
            #optimize_model()

            # Compute a mask of non-final states and concatenate the batch elements
            non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                                batch.next_state)), dtype=torch.bool)
            non_final_next_states = torch.cat([s for s in batch.next_state
                                                        if s is not None])
            state_batch = torch.cat(batch.state)
            action_batch = torch.cat(batch.action)
            reward_batch = torch.cat(batch.reward)

            # Compute Q(s_t, a) - the model computes Q(s_t), then we select the columns of actions taken
            # That means that for each state in the batch, we receive the action that we would have taken according to the policy
            state_action_values = q_network(state_batch).gather(1, action_batch)

            # Here, we compute the next state values accordin gto the target network
            next_state_values = torch.zeros(BATCH_SIZE)
            with torch.no_grad():
                next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
            expected_state_action_values = (next_state_values * GAMMA) + reward_batch

            # Compute Huber loss
            criterion = nn.SmoothL1Loss()
            loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

            # Optimize the model
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(q_network.parameters(), 100)
            optimizer.step()

            # Update the target network
            target_net_state_dict = target_net.state_dict()
            q_net_state_dict = q_network.state_dict()
            for key in q_net_state_dict:
                target_net_state_dict[key] = q_net_state_dict[key] * TAU + target_net_state_dict[key] * (1-TAU)
            target_net.load_state_dict(target_net_state_dict)

        if done:
            plot_rewards()
            episode_rewards.append(episode_reward)
            break

plot_rewards(show_result=True)
plt.ioff()
plt.show()

In [ ]:
# visualize the behavious of the agent
observation, info = env.reset()

for _ in range(1000):
    action = q_network(observation).max(1).indices.view(1,1)
    observation, reward, terminated, truncated, _ = env.step(action.item())

## DRL with stable_baselines3

Luckily, we don't have to implement all standard DRL algorithms ourselves when we want to use them. There are libraries, such as ```stable_baselines3```, that offer many different ready-to-use DRL algorithms, including DQN, which we just implemented.

In the following, we check out at how to train with the ```stable_baselines3``` implementation of DQN, and then use that knowledge to train an other commonly used DRL algorithm, called PPO. Please check out the ```stable_baselines3``` [DQN documentation](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html).

We will not go into the details of PPO here, but please feel free to read up on the ```stable_baselines3``` [PPO documentation](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html) or take a look at the [original paper](https://arxiv.org/abs/1707.06347).

In [ ]:
# First, we import the libraries: gymnasium for the environment, stable_baselines3 for algorithms
import gymnasium as gym
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import configure, read_csv

In [ ]:
# Creating the environment + agent
env = gym.make("CartPole-v1", render_mode="human")
dqn_model = DQN("MlpPolicy", env, verbose=1)

# To get an easier access to evaluation data, configure the logger to write to a file
dqn_tmp_path = "tmp/sb3_log/dqn"
new_dqn_logger = configure(dqn_tmp_path, ["csv"])
dqn_model.set_logger(new_dqn_logger)

# Training and saving the agent
dqn_model.learn(total_timesteps=50000, log_interval=4)
dqn_model.save("dqn_cartpole")


In [ ]:
# Model loading and evaluation
dqn_model = DQN.load("dqn_cartpole")
dqn_mean_rew, dqn_std_rew = evaluate_policy(dqn_model, env, n_eval_episodes=10)
print(f"DQN evaluation results - mean reward: {dqn_mean_rew}, std. deviation: {dqn_std_rew}")

In [ ]:
# Now, let's look at the training development, by examining the logger data.
# read the logger output
dqn_train_log = read_csv("tmp/sb3_log/progress.csv")
# take a quick look at the data...
dqn_train_log

In [ ]:
# Let's plot some of the training evaluation!
import matplotlib.pyplot as plt

# First, we plot the reward development in training
plt.title("DQN mean reward during training")
plt.plot(dqn_train_log['time/total_timesteps'], dqn_train_log['rollout/ep_rew_mean'], label="dqn")
plt.legend()
plt.xlabel("# timesteps")
plt.ylabel("mean reward")
plt.show()

# Let's also look at the loss
plt.title("DQN training loss")
plt.plot(dqn_train_log['time/total_timesteps'], dqn_train_log['train/loss'], label="dqn")
plt.legend()
plt.xlabel("# timesteps")
plt.ylabel("training loss")
plt.show()

Now it is your turn! Fill in the code below to train, save, load, and evaluate a PPO model on the cartpole task.

In [ ]:
# TODO
""" START CODE """
# Create the CartPole environment


# Create a PPO agent


# As with DQN, we save the logger output to a file.



# Train the PPO agent for 50000 timesteps (feel free to try more/less!)


# Save the trained model to a file for later use!

""" END CODE """


In [ ]:
# Now, we want to evaluate the trained model that we saved earlier!

# First, let's load the model
ppo_model = PPO.load("ppo_cartpole")

# Use evaluate_policy to receive the mean and standard reward of the trained policy over e.g. 10 episodes.
ppo_mean_rew, ppo_std_rew = evaluate_policy(ppo_model, env, n_eval_episodes=10)

# Print the results
print(f"PPO evaluation results - mean reward: {ppo_mean_rew}, std. deviation: {ppo_std_rew}")

In [ ]:
# TODO: Let's do some visualization again! For that, we consult the log file again.
# read the logger output
""" START CODE """

""" END CODE """
# take a quick look at the data...
""" START CODE """

""" END CODE """


In [ ]:
# TODO: Let's plot the PPO results like before!
# You can re-use the code from above here and add the PPO log data.
# First, we plot the reward development in training
""" START CODE """


""" END CODE """

That's it!

Now you saw how easy it is to train DRL models with libraries like ```stable_baselines3```.

## Safe RL with omnisafe and safety_gymnasium

An important challenge (and a very wide topic) in RL research is safety. In this part of the notebook, we look at two libraries that allow us to easily train save RL agents on suitable environments.

[```safety_gymnasium```](https://safety-gymnasium.readthedocs.io/en/latest/index.html) provides environments that include notions of costs and constraints. In our task, we will use the ```SafetyCarCircle1-v0``` environment - that means a safety environment, with a Car agent, for the Circle Task, Level 1. Check out the documentation on the Circle task [here](https://safety-gymnasium.readthedocs.io/en/latest/environments/safe_navigation/circle.html#circle1)!

In this Circle task, the agent learns to drive in circles around the center of a circled area. Level 0 does not implement any constraints, so it is not interesting to us here, but level 1 does! In level one, boundaries are implemented on two sides of the circle, and the agent needs to avoid crossing these boundaries. If it does, it is punished by receiving a cost.

Additionally, the [```omnisafe```](https://www.omnisafe.ai/en/latest/index.html) library provides implementation of some DRL algorithms, like PPO, which we looked at in the previous section, as well as save versions of these algorithms. Here, we will use PPO, and the safe algorithm PPOLag, which implements Lagrangian constraints. Again, we do not need to understand this in detail for this notebook, but if you are interested, please read the omnisafe [documentation](https://www.omnisafe.ai/en/latest/benchmark/on-policy.html), and also consider the original papers of [PPO](https://arxiv.org/pdf/1707.06347) and the Lagrangian version [PPOLag](https://cdn.openai.com/safexp-short.pdf).

In [ ]:
# First, we import the necessary libraries
import omnisafe
import safety_gymnasium

In [ ]:
# create the environment
ENV_ID = 'SafetyCarCircle1-v0'
env = safety_gymnasium.make(ENV_ID,
                            render_mode='human')

# We set some configurations: how many steps we want to train for, and how many steps there are in an episode
# The number of steps per episode can be found in the Circle task documentation.
# The documentation on which configuration settings are relevant is unfortunately a bit unclear - these two seem to be the minimum.
# Feel free to figure out more!
cfg = {
    'train_cfgs': {
        'total_steps': 100000,
    },
    'algo_cfgs': {
        'steps_per_epoch': 500,
    }
}

### Collapse/view training log for PPO agent
Output log during training is very long. Collapse to not have to scroll down super far.

In [ ]:
# Now, we create an omnisafe Agent: We want to train the PPO algorithm, in the specified environment, with the specified configuration
agent = omnisafe.Agent('PPO', ENV_ID, custom_cfgs=cfg)
# The only thing we need to do to start training is call this function!
agent.learn()

### Collapse/view training log for PPOLag agent

In [ ]:
# Now you get to do the same thing, but for Safe RL training!
# TODO: Create a safe RL agent that trains the PPOLag algorithm in the specified environment, with the same configuration data
# Then, run the training!
""" START CODE """


""" END CODE """

### omnisafe evaluation

In [ ]:
# Evaluation in omnisafe is also rather simple!
# agent.plot() will plot the development of rewards and costs over the training steps
agent.plot()
# agent.evaluate() will evaluate the agent, and output the average cost and reward.
agent.evaluate(num_episodes=1)

In [ ]:
# Now it's your turn again!
# TODO: Plot and evaluate the safe agent.
""" YOUR CODE """

""" END CODE"""

In [ ]:
# We can also render an episode of the trained agent by calling agent.render().
agent.render(num_episodes=1)

In [ ]:
# TODO: Now, render an episode for the safe agent!
""" START CODE """

""" END CODE """